In [ ]:
#this is group work
# import the necessary packages
from __future__ import print_function
from imutils.object_detection import non_max_suppression
from imutils import paths
import numpy as np
import imutils
import cv2

# initialize the HOG descriptor/person detector
hog = cv2.HOGDescriptor()
hog.setSVMDetector(cv2.HOGDescriptor_getDefaultPeopleDetector())

# loop over the image paths
imagePaths = list(paths.list_images(r"C:\Users\Emyr\Documents\Jupyter\pedestrian-detection\images"))

for imagePath in imagePaths:
    # load the image and resize it to (1) reduce detection time
    # and (2) improve detection accuracy
    image = cv2.imread(imagePath)
    print (image.shape[0])
    print (image.shape[1])
    print (image.shape[2])
    image = imutils.resize(image, width=min(6400, image.shape[1]))
    orig = image.copy()

    # detect people in the image
    (rects, weights) = hog.detectMultiScale(image, winStride=(4, 4), padding=(8, 8), scale=1.05)

    # draw the original bounding boxes
    for (x, y, w, h) in rects:
        cv2.rectangle(orig, (x, y), (x + w, y + h), (0, 0, 255), 2)

    # apply non-maxima suppression to the bounding boxes using a
    # fairly large overlap threshold to try to maintain overlapping
    # boxes that are still people
    rects = np.array([[x, y, x + w, y + h] for (x, y, w, h) in rects])
    pick = non_max_suppression(rects, probs=None, overlapThresh=0.65)

    # draw the final bounding boxes
    for (xA, yA, xB, yB) in pick:
        cv2.rectangle(image, (xA, yA), (xB, yB), (0, 255, 0), 2)

    # show some information on the number of bounding boxes
    filename = imagePath[imagePath.rfind("/") + 1:]
    print("[INFO] {}: {} original boxes, {} after suppression".format(filename, len(rects), len(pick)))

    # show the output images
    cv2.imshow("Before NMS", orig)
    cv2.imshow("After NMS", image)
    cv2.waitKey(0)




In [1]:
# import the necessary packages
from __future__ import print_function
from imutils.object_detection import non_max_suppression
from imutils import paths
import numpy as np
import imutils
import cv2
import sys

# initialize the HOG descriptor/person detector
hog = cv2.HOGDescriptor()
hog.setSVMDetector(cv2.HOGDescriptor_getDefaultPeopleDetector())

# loop over the image paths

caps = cv2.VideoCapture(r'C:/Users/Emyr/Documents/Jupyter/pedestrian-detection/video/Pedestrians.mp4')
ret, frame = caps.read()

while(ret):
    ret, frame = caps.read()
    
    frame = imutils.resize(frame, width = min(frame.shape[0], frame.shape[1]))
    orig = frame.copy()

    # detect people in the image
    (rects, weights) = hog.detectMultiScale(frame, winStride=(2, 2), padding=(8, 8), scale=0.5) #scale-1.05

    # draw the original bounding boxes
    for (x, y, w, h) in rects:
        cv2.rectangle(orig, (x, y), (x + w, y + h), (0, 0, 255), 2)

    # apply non-maxima suppression to the bounding boxes using a
    # fairly large overlap threshold to try to maintain overlapping
    # boxes that are still people
    rects = np.array([[x, y, x + w, y + h] for (x, y, w, h) in rects])
    pick = non_max_suppression(rects, probs=None, overlapThresh=0.65)

    # draw the final bounding boxes
    for (xA, yA, xB, yB) in pick:
      
        cv2.rectangle(frame, (xA, yA), (xB, yB), (0, 255, 0), 2)
        print(xA, yA, xB, yB)
    cv2.imshow("After NMS", frame)
    cv2.waitKey(1) 
    

caps.release()
cv2.destroyAllWindows()



623 145 687 273
622 145 686 273
622 145 686 273
622 145 686 273
622 144 686 272
622 144 686 272
578 74 642 202
581 67 645 195
464 0 528 120
463 0 527 120
463 0 527 121
462 0 526 121
461 0 525 121
461 0 525 122
460 0 524 123
461 0 525 124
460 0 524 126
459 0 523 126
458 0 522 126
640 0 704 121
455 0 519 128
640 0 704 120
455 1 519 129
454 0 518 128
453 1 517 129
451 3 515 131
440 11 504 139
439 11 503 139
437 13 501 141
436 14 500 142
435 17 499 145
435 19 499 147
434 22 498 150
434 23 498 151
434 23 498 151
433 24 497 152
431 25 495 153
431 25 495 153
430 25 494 153
429 25 493 153
427 26 491 154
426 26 490 154
426 26 490 154
421 37 485 165
420 38 484 166
419 39 483 167
415 39 479 167
414 39 478 167
413 39 477 167
412 39 476 167
411 41 475 169
411 43 475 171
410 45 474 173
410 47 474 175
408 50 472 178
407 51 471 179
406 53 470 181
404 55 468 183
402 58 466 186
402 58 466 186
401 59 465 187
400 60 464 188
398 60 462 188
396 61 460 189
394 62 458 190
393 63 457 191
392 64 456 192
391 66 

248 20 312 148
401 43 465 171
247 22 311 150
401 42 465 170
246 24 310 152
401 40 465 168
246 25 310 153
401 38 465 166
244 27 308 155
402 35 466 163
242 29 306 157
403 34 467 162
241 30 305 158
403 33 467 161
240 30 304 158
404 33 468 161
237 31 301 159
417 3 481 131
405 34 469 162
235 32 299 160
405 33 469 161
234 33 298 161
418 2 482 130
233 33 297 161
406 33 470 161
418 1 482 129
231 34 295 162
407 33 471 161
229 35 293 163
408 29 472 157
227 36 291 164
409 27 473 155
410 25 474 153
411 24 475 152
413 21 477 149
218 46 282 174
414 20 478 148
216 46 280 174
414 19 478 147
214 47 278 175
414 19 478 147
212 48 276 176
415 18 479 146
211 48 275 176
416 18 480 146
210 48 274 176
416 18 480 146
210 48 274 176
416 18 480 146
207 50 271 178
417 15 481 143
206 50 270 178
417 16 481 144
206 50 270 178
418 14 482 142
206 51 270 179
418 14 482 142
419 13 483 141
419 11 483 139
420 10 484 138
420 8 484 136
420 5 484 133
194 65 258 193
420 5 484 133
193 66 257 194
420 4 484 132
191 67 255 195
42

KeyboardInterrupt: 